# 1. Example of modification attack (Graph Classification)

[GRB](https://cogdl.ai/grb/home) supports modification attack on graph classification attack.

Contents
- [Load Datasets](##Load-Datasets)
- [Prepare Model](##Prepare-Model)
- [Modification Attack](##Modification-Attack)

In [1]:
import os
import torch
import numpy as np
import scipy.sparse as sp
import grb.utils as utils
from grb.dataset import CogDLDataset, OGBDataset

## 1.1. Load Datasets

In [12]:
dataset_name = "mutag"
data_dir="../../data/"

dataset = CogDLDataset(name=dataset_name, data_dir=data_dir)

Dataset 'mutag' loaded.
    Number of graphs: 188
    Number of nodes (maximum): 28
    Number of edges (maximum): 66
    Number of features: 7
    Number of classes: 2
    Number of train samples: 150
    Number of val samples: 18
    Number of test samples: 20


## 1.2. Prepare Model

GRB supports models based on pure Pytorch, CogDL or DGL. The following is an example of GCNGC (GCN for Graph Classification) implemented by pure Pytorch. Other models can be found in ``grb/model/torch``, ``grb/model/cogdl``, or ``grb/model/dgl``.

### 1.2.1. GCNGC (Graph Convolutional Network for Graph Classification)

In [13]:
from grb.model.torch import GCNGC

model_name = "gcngc"
model = GCNGC(in_features=dataset.num_features,
              out_features=dataset.num_classes,
              hidden_features=64, 
              n_layers=3,
              residual=False,
              dropout=0.5)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 4802.
GCNGC(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=7, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GCNConv(
      (linear): Linear(in_features=64, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
  )
  (linear): Linear(in_features=64, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


### 1.2.2 Training

GRB provides ``grb.trainer.trainer`` that facilitates the training process of GNNs. For Graph Classification task, a mini-batch training on graphs is applied. Multiple graphs are merged into a large graph, then the results are pooled to predict label for each graph.

In [14]:
save_dir = "./saved_models/{}/{}".format(dataset_name, model_name)
save_name = "model.pt"
device = "cuda:0"
batch_size = 20

In [15]:
from grb.trainer.trainer import GraphTrainer

trainer = GraphTrainer(dataset=dataset, 
                       batch_size=batch_size,
                       optimizer=torch.optim.Adam(model.parameters(), lr=0.01),
                       loss=torch.nn.functional.cross_entropy,
                       lr_scheduler=False,
                       early_stop=True,
                       early_stop_patience=50,
                       device=device)

In [16]:
trainer.train(model=model,
              n_epoch=200,
              eval_every=1,
              save_after=0,
              save_dir=save_dir,
              save_name=save_name,
              verbose=False)

  0%|          | 0/200 [00:00<?, ?it/s]

Training finished. Best validation score: 0.9444
Training runtime: 10.7984.


### 1.2.3 Inference

In [17]:
model = torch.load(os.path.join(save_dir, save_name))
model = model.to(device)
model.eval()

GCNGC(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=7, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GCNConv(
      (linear): Linear(in_features=64, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
  )
  (linear): Linear(in_features=64, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [18]:
# by trainer
pred = trainer.inference(model)

### 1.2.4 Evaluation (without attack)

In [19]:
# by trainer
test_score = trainer.evaluate(model, dataset.index_test)
print("Test score: {:.4f}".format(test_score))

Test score: 0.7500


## 1.3. Modification Attack

In [48]:
from grb.attack.modification.flip import FLIP

In [49]:
adj_attack_list = []
for i in dataset.index_test:
    print("Attacking graph {}".format(i))
    graph = dataset.graphs[i]
    adj = utils.build_adj(graph.edge_attr, graph.edge_index)
    n_edge_test = adj.getnnz()
    n_mod_ratio = 0.5
    n_edge_mod = int(n_edge_test * n_mod_ratio)
    # degree flipping
    attack = FLIP(n_edge_mod, flip_type="deg", mode="descend", device=device)
    adj_attack = attack.attack(adj, index_target=np.arange(graph.num_nodes))
    adj_attack_list.append(adj_attack)

Attacking graph 124


  0%|          | 0/44 [00:00<?, ?it/s]

FLIP attack finished. 22 edges were flipped.
Attacking graph 47


  0%|          | 0/38 [00:00<?, ?it/s]

FLIP attack finished. 19 edges were flipped.
Attacking graph 145


...

FLIP attack finished. 15 edges were flipped.
Attacking graph 116


  0%|          | 0/44 [00:00<?, ?it/s]

FLIP attack finished. 22 edges were flipped.
Attacking graph 115


  0%|          | 0/20 [00:00<?, ?it/s]

FLIP attack finished. 10 edges were flipped.


In [60]:
logits =  torch.zeros((len(adj_attack_list), dataset.num_classes)).to(device)
for i in range(len(adj_attack_list)):
    adj = utils.adj_preprocess(adj_attack_list[i], device=device)
    logits[i] = model(dataset.graphs[dataset.index_test[i]].x.to(device), adj)
score = trainer.eval_metric(logits, dataset.labels[dataset.index_test].to(device))
print("Test score (after attack): {:.4f}".format(score))

Test score (after attack): 0.6000


For further information, please refer to the [GRB Documentation](https://grb.readthedocs.io/en/latest/).